## QHyper usecase

### Create instance of problem

In [1]:
from QHyper.problems.knapsack import KnapsackProblem
from QHyper.problems.tsp import TSPProblem

PROBLEM_TYPE = 'knapsack' # 'tsp'


# Each problem needs different parameters, because it depends on the number
# of variables and constraints
# Number of hyper_optimizer_bounds depends on the number of constraints,
# because each constraint requires one weights, and objective function also
# requires one
 
if PROBLEM_TYPE == 'knapsack':
    # Create knapsack with 3 items: 2 with weights 1 and value 2 - (1, 2),
    # and one with weight 1 and value 1 - (1, 1)
    problem = KnapsackProblem(max_weight=2, items=[(1, 2), (1, 2),(1, 1)])
    print(problem.knapsack.items)
    
    params_cofing = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2.5, 2.5],
    }
    hyper_optimizer_bounds = 3*[(1, 10)]

elif PROBLEM_TYPE == 'tsp':
    # Create Traveling Salesmam Problem with 3 cities
    problem = TSPProblem(
        number_of_cities=3,
    )
    params_cofing = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2, 2, 2, 2],
    }
    hyper_optimizer_bounds = 5*[(1, 10)]

[Item(weight=1, value=2), Item(weight=1, value=2), Item(weight=1, value=1)]


In [2]:
print(f"Variables used to describe objective function"
      f"and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints:")
for constraint in problem.constraints:
    print(f"    {constraint}")

Variables used to describe objective functionand constraints: (x0, x1, x2, x3, x4)
Objective function: {('x0',): -2, ('x1',): -2, ('x2',): -1}
Constraints:
    {('x3',): -1, ('x4',): -1, (): 1}
    {('x0',): -1, ('x1',): -1, ('x2',): -1, ('x3',): 1, ('x4',): 2}


### Use VQA to solve knapsack problem

In [3]:
from QHyper.solvers.vqa.base import VQA
from QHyper.solvers import Solver

In [4]:
# Simple quantum circuit without optimzers will be used to test the results
# WF-QAOA is choosen becasue this PQC has most suitable evaluation function

# tester_config = {
#     'pqc': {
#         'type': 'wfqaoa',
#         'layers': 5,
#     }
# }

# tester = VQA(problem, config=tester_config)

In [5]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)

solver_config = {
    "solver": {
        "type": "vqa",
        "args": {
            "config": {
                "optimizer": {
                    "type": "scipy",
                    "maxfun": 200,
                },
                "pqc": {
                    "type": "wfqaoa",
                    "layers": 5,
                    "limit_results": 20,
                    "penalty": 2
                }
            }
        }
    }
}

vqa = Solver.from_config(problem, solver_config)

# # 2. Providing actual isntance of each class like VQA and Optimizer
# from QHyper.solvers.vqa.pqc.h_qaoa import HQAOA
# from QHyper.optimizers import ScipyOptimizer

# vqa = VQA(problem, HQAOA(layers=5), ScipyOptimizer(maxfun=200))

In [6]:
# Run VQA with provided initial parameters
solver_results = vqa.solve(params_cofing)

print("Solver results:")
print(f"Probabilities: {solver_results.results_probabilities}")
print(f"Best params: {solver_results.params}")

Best hyper args: [1.  2.5 2.5]
Solver results:
Probabilities: {'00000': 0.0732912838324004, '00001': 0.01812365507384847, '00010': 0.05407207652748644, '00011': 0.0701062245930107, '00100': 0.017498889261866067, '00101': 0.009292651366578255, '00110': 0.050293202934572685, '00111': 0.016844666311210565, '01000': 0.046624559969783416, '01001': 0.009582944060964814, '01010': 0.04374581517898236, '01011': 0.007336159588485337, '01100': 0.010700779894629564, '01101': 0.07661867874108277, '01110': 0.007507066502992263, '01111': 0.023008509073494634, '10000': 0.04662455996978336, '10001': 0.0095829440609648, '10010': 0.04374581517898232, '10011': 0.00733615958848532, '10100': 0.010700779894629532, '10101': 0.07661867874108272, '10110': 0.00750706650299228, '10111': 0.02300850907349465, '11000': 0.02873850917070903, '11001': 0.06522050516274285, '11010': 0.01700164098761874, '11011': 0.02196157538990109, '11100': 0.035843090675343484, '11101': 0.014437271029358385, '11110': 0.0024948049870106

In [7]:
from QHyper.util import weighted_avg_evaluation, sort_solver_results

# Evaluate results with weighted average evaluation
print("Evaluation:")
print(weighted_avg_evaluation(
    solver_results.results_probabilities, problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sorted results:")
print(sort_solver_results(solver_results.results_probabilities, limit_results=10))

Evaluation:
-1.2555153432454813
Sorted results:
{'01101': 0.07661867874108277, '10101': 0.07661867874108272, '00000': 0.0732912838324004, '00011': 0.0701062245930107, '11001': 0.06522050516274285, '11111': 0.054530926675500725, '00010': 0.05407207652748644, '00110': 0.050293202934572685, '01000': 0.046624559969783416, '10000': 0.04662455996978336}


In [8]:
# Here created ealier tester will be used to evaluate results

# best_results = tester.evaluate(best_params)
# print(f"Best results: {best_results}")
# print(f"Params used for optimizer:\n{best_params['angles']},\n"
#       f"and params used for hyper optimizer: {best_params['hyper_args']}")

#### Using hyper optimizers

In [9]:
# Additionally other optimizer can be used to tune some parameters, in below
# example, Random optimzier will change weights (hyper_args) and choose ones
# that gives the best results after runnign 200 iteration of scipy minimizer

solver_config = {
    "solver": {
        "type": "vqa",
        "args": {
            "config": {
                "optimizer": {
                    "type": "scipy",
                    "maxfun": 200,
                },
                "pqc": {
                    "type": "wfqaoa",
                    "layers": 5,
                    "limit_results": 20,
                    "penalty": 2
                }
            }
        }
    },
    "hyper_optimizer": {
        "type": "random",    
        "processes": 5,
        "number_of_samples": 100,
        "bounds": hyper_optimizer_bounds
    }
}
vqa = Solver.from_config(problem, solver_config)

In [10]:
solver_results = vqa.solve(params_cofing)

print("Solver results:")
print(f"Probabilities: {solver_results.results_probabilities}")
print(f"Best params: {solver_results.params}")

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [05:44<00:00,  3.44s/it]


Best hyper args: [2.04005423 7.83521531 6.33957791]
Solver results:
Probabilities: {'00000': 0.059239354303146786, '00001': 0.011548323289243477, '00010': 0.032373346567151184, '00011': 0.004432339559983008, '00100': 0.003227543534714331, '00101': 0.014650296991314817, '00110': 0.12355778019742689, '00111': 0.0076988056892276086, '01000': 0.005376032149061929, '01001': 0.010237226411964223, '01010': 0.10710115653854947, '01011': 0.008860764441300541, '01100': 0.008615873127187847, '01101': 0.10812773570267861, '01110': 0.007405266528697867, '01111': 0.006771485701362549, '10000': 0.005376032149061932, '10001': 0.010237226411964217, '10010': 0.10710115653854947, '10011': 0.008860764441300528, '10100': 0.008615873127187855, '10101': 0.10812773570267857, '10110': 0.007405266528697866, '10111': 0.006771485701362545, '11000': 0.008442885983484154, '11001': 0.09616717030614107, '11010': 0.0108197025868455, '11011': 0.0062708363355956035, '11100': 0.00897497056675387, '11101': 0.0160414379317

In [11]:
print("Evaluation:")
print(weighted_avg_evaluation(
    solver_results.results_probabilities, problem.get_score,
    penalty=0, limit_results=20, normalize=True
))
print("Sorted results:")
print(sort_solver_results(solver_results.results_probabilities, limit_results=10))

Evaluation:
-1.719135691005536
Sorted results:
{'00110': 0.12355778019742689, '01101': 0.10812773570267861, '10101': 0.10812773570267857, '01010': 0.10710115653854947, '10010': 0.10710115653854947, '11001': 0.09616717030614107, '11111': 0.061790068671762725, '00000': 0.059239354303146786, '00010': 0.032373346567151184, '11101': 0.016041437931758287}
